In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws.get_details()

In [ ]:
# see what your run definition looks like
with open(".azureml/explain.runconfig","r") as f:
    print(f.read())

In [ ]:
# see what your package dependencies looks like
with open(".azureml/conda_dependencies.yml","r") as f:
    print(f.read())

In [24]:
#submit and monitor a run
from azureml.core import RunConfiguration, Experiment
from azureml.core.script_run_config import ScriptRunConfig
from azureml.widgets import RunDetails

runconfig = RunConfiguration.load('.','explain')
src = ScriptRunConfig(source_directory='.',script='train_explain.py',run_config=runconfig)
run = Experiment(ws,'generate-attrition-explainer').submit(src)
RunDetails(run).show()

In [ ]:
#see what your score file looks like
with open("score.py","r") as f:
    print(f.read())

In [ ]:
#deploy
from azureml.core import Workspace
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig, Model

ws = Workspace.from_config()

scoring_explainer_model = Model(ws, 'IBM_attrition_explainer')
attrition_model = Model(ws, 'IBM_attrition_model')

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "IBM_Attrition",  
                                                     "method" : "local_explanation"}, 
                                               description='Explain predictions on employee attrition')
                                               
inference_config = InferenceConfig(entry_script='score.py', 
                                   extra_docker_file_steps='Dockerfile', 
                                   runtime='python', 
                                   conda_file='myenv.yml')

service = Model.deploy(ws, name='predictattritionsvc', models=[scoring_explainer_model, attrition_model], inference_config= inference_config, deployment_config=aciconfig)

In [ ]:
# get URI for querying
from azureml.core.webservice import AciWebservice
import requests
import json

sample = '{"Age":{"899":49},"BusinessTravel":{"899":"Travel_Rarely"},"DailyRate":{"899":1098},"Department":{"899":"Research & Development"},"DistanceFromHome":{"899":4},"Education":{"899":2},"EducationField":{"899":"Medical"},"EnvironmentSatisfaction":{"899":1},"Gender":{"899":"Male"},"HourlyRate":{"899":85},"JobInvolvement":{"899":2},"JobLevel":{"899":5},"JobRole":{"899":"Manager"},"JobSatisfaction":{"899":3},"MaritalStatus":{"899":"Married"},"MonthlyIncome":{"899":18711},"MonthlyRate":{"899":12124},"NumCompaniesWorked":{"899":2},"OverTime":{"899":"No"},"PercentSalaryHike":{"899":13},"PerformanceRating":{"899":3},"RelationshipSatisfaction":{"899":3},"StockOptionLevel":{"899":1},"TotalWorkingYears":{"899":23},"TrainingTimesLastYear":{"899":2},"WorkLifeBalance":{"899":4},"YearsAtCompany":{"899":1},"YearsInCurrentRole":{"899":0},"YearsSinceLastPromotion":{"899":0},"YearsWithCurrManager":{"899":0}}'

service = AciWebservice(ws,"predictattritionsvc")
print(service.scoring_uri)

headers = {'Content-Type':'application/json'}

# send request to service
resp = requests.post(service.scoring_uri, sample, headers=headers)
print("prediction:", resp.text)
result = json.loads(resp.text)

In [ ]:
#plot the feature importance for the prediction
import numpy as np
import matplotlib.pyplot as plt; plt.rcdefaults()

labels = json.loads(sample)
labels = labels.keys()
objects = labels
y_pos = np.arange(len(objects))
performance = result["local_importance_values"][0][0]

plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
locs, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.ylabel('Feature impact - leaving vs not leaving')
plt.title('Local feature importance for prediction')

plt.show()